In [3]:
import numpy as np
import os
import io
import PIL
import PIL.Image
import PIL.ImageOps
from PIL import ImageFont
from PIL import Image
from PIL import ImageDraw
from PIL import ImageOps
from PIL import ImageColor
import tensorflow as tf

import matplotlib.pyplot as plt
from keras.layers import *
import time
import tempfile
from urllib.request import urlopen
from io import BytesIO
from keras.preprocessing import image
import numpy as np 
import urllib
import cv2 
import os

In [4]:
train_dataset = tf.keras.utils.image_dataset_from_directory("DatasetOOT/Training/",
                                      batch_size=10, shuffle = True, image_size=(300,300))

Found 198 files belonging to 7 classes.


In [5]:
validation_dataset = tf.keras.utils.image_dataset_from_directory("DatasetOOT/Validation/",
                                                          batch_size=4, image_size=(300,300))

Found 52 files belonging to 7 classes.


In [6]:
train_dataset.class_names
class_names = train_dataset.class_names
print(class_names)

['8719992763139', '8719992763351', '8719992763511', '8719992763542', 'CrunchyNuts', 'KetoGranola', 'KoffieKeto']


In [8]:
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",input_shape=(300,300,3)),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomZoom(-0.1),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.GaussianNoise(2),
    tf.keras.layers.GaussianNoise(1),
    tf.keras.layers.RandomBrightness([-0.2, 0.5])
  ]
)

best_callbacks = [tf.keras.callbacks.EarlyStopping(patience = 1000) ,
                 tf.keras.callbacks.ModelCheckpoint(filepath="OOTTrainedModel/",monitor="val_accuracy",
                    save_best_only=True,
                    save_weights_only=False,
                    mode="auto",
                    save_freq="epoch",)]

In [9]:
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters, kernel_size=(3, 3)):
        super(ConvBlock, self).__init__()
        self.conv = Conv2D(num_filters, kernel_size)
        self.bn = BatchNormalization()
        self.elu = Activation("elu")
        self.pooling = MaxPool2D((2, 2))

    def call(self, x, pool=True):
        y = self.conv(x)
        x = self.bn(y)
        x = self.elu(x)
        x = Add()([x,y])
        x = self.elu(x)
        if pool == True:
            x = self.pooling(x)
            
        return x

In [13]:
def build_model(input_shape):
    inputs = Input(input_shape)
    inputs = data_augmentation(inputs)
    x = ConvBlock(32)(inputs)
    x = ConvBlock(64)(x)
    x = ConvBlock(300)(x, pool=False)
    x = ConvBlock(300)(x, pool=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.2)(x)
    x = Dense(2, activation="softmax")(x)
    
    model = tf.keras.models.Model(inputs, x)
    return model

In [14]:
model = build_model((300,300, 3))

In [15]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=(0.001),
                                                 beta_1=(0.9),
                                                 beta_2=(0.999),
                                                 epsilon=(1e-7),),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [16]:
model.fit = model.fit(train_dataset,epochs=180, 
                      validation_data=validation_dataset, 
                      shuffle=True)

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "c:\Users\Sua.Wilhelm\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\Sua.Wilhelm\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\Sua.Wilhelm\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\Sua.Wilhelm\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\Sua.Wilhelm\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\Sua.Wi

In [ ]:
model.summary()

In [ ]:
best_model_path = "OOTTrainedModel/"

best_model = tf.keras.models.load_model(best_model_path)

test_loss, test_acc = best_model.evaluate(validation_dataset)
print(test_acc)
y_prob = best_model.predict(validation_dataset)
y_classes = y_prob.argmax(axis=-1)
print(y_classes)
print( y_prob.argmax(axis=1)[1])

In [ ]:
dir_path = "DatasetOOT/Test/"

for i in os.listdir(dir_path):
    img = tf.keras.preprocessing.image.load_img(dir_path+"//" + i, target_size=(300,300),color_mode="grayscale")
    plt.imshow(img)
    plt.show()
    
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    images = np.vstack([x])
    val = best_model.predict(images)
    if val.argmax(axis=1) == 0:
        print("crunchy nuts")
    else:
        print(val.argmax(axis=1))